## Extensões na Prática

- O que são extensões?
- Documentação [https://duckdb.org/docs/extensions/overview.html](https://duckdb.org/docs/extensions/overview.html)
- Oficiais [https://duckdb.org/docs/extensions/official_extensions](https://duckdb.org/docs/extensions/official_extensions)
- Hands On

In [ ]:
!pip install duckdb --quiet
!pip install pandas --quiet 

In [ ]:
!pip install magic_duckdb jupysql 
%load_ext magic_duckdb
%config SqlMagic.autopandas = True

In [ ]:
%dql select  * from  (values (1, 'a'), (2, 'b')) as t(a, b);

In [ ]:
%%time 
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)
query = con.execute("SELECT *  FROM nyc_table limit 10  ").fetch_arrow_table()
query.to_pandas()

In [ ]:
#autocompleteno cli 

## AWS 

- INSTALL aws;
- INSTALL httpfs;
- LOAD aws;
- LOAD httpfs;
    - [Referência](https://duckdb.org/docs/extensions/httpfs/s3api.html)

In [ ]:
%%dql

CREATE SECRET secretGCP (
    TYPE GCS,
    KEY_ID 'GOO****7F5',
    SECRET '*****'
);

SELECT * FROM read_parquet('gcs://lance-db/teste.parquet');

In [ ]:
%%dql

SELECT * FROM read_parquet('gcs://lance-db/teste.parquet');

## httpfs 
- INSTALL httpfs;
- LOAD httpfs;

In [ ]:
%dql

select * from 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet' limit 10;

SELECT * FROM read_parquet(['https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet','https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet']);

## Azure
- [Referência](https://duckdb.org/docs/extensions/azure)


In [ ]:
%dql
INSTALL azure;
LOAD azure;

CREATE SECRET secret1 (
    TYPE AZURE,
    CONNECTION_STRING '⟨value⟩'
);


SELECT count(*)
FROM 'az://⟨my_container⟩/⟨path⟩/⟨my_file⟩.⟨parquet_or_csv⟩';


SELECT count(*)
FROM 'az://⟨my_storage_account⟩.blob.core.windows.net/⟨my_container⟩/⟨path⟩/⟨my_file⟩.⟨parquet_or_csv⟩';


SELECT count(*)
FROM 'abfss://⟨my_filesystem⟩/⟨path⟩/⟨my_file⟩.⟨parquet_or_csv⟩';

SELECT count(*)
FROM 'abfss://⟨my_filesystem⟩/⟨path⟩/⟨my_file⟩.⟨parquet_or_csv⟩';




## Delta

- [Referência](https://duckdb.org/docs/extensions/delta)
- Nativo a partir da versão 1.0

In [ ]:
!pip install deltalake --quiet
!pip install pyarrow --quiet
!pip install pandas --quiet
!pip install duckdb --quiet

In [ ]:

import pandas as pd
from deltalake import DeltaTable, write_deltalake
from pyarrow import dataset as ds
import duckdb
acsv = ds.dataset('data/aula4/yellow_tripdata_2017-01.parquet')
working_columns = ["VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","airport_fee"]
dataset_scanner = acsv.scanner(columns=working_columns)
nyc_table = dataset_scanner.to_table()

con = duckdb.connect()
query = con.execute("SELECT *  FROM nyc_table limit 10  ")
df = query.df()

write_deltalake('data/aula4/delta', df, mode='overwrite')



In [ ]:
con.execute("INSTALL delta; ")
con.execute("LOAD delta; ")
query = con.execute("SELECT * FROM delta_scan('data/aula4/delta') ORDER BY 1; ")
df = query.df()
df

## Spatial

    - [Kepler](https://kepler.gl/demo)

In [ ]:
#install duckdb version 0.10
!pip install duckdb --quiet
!pip install geopandas --quiet 
!pip install mapclassify --quiet 

In [ ]:

import duckdb
import geopandas as gpd
import pandas as pd 

In [ ]:
con = duckdb.connect()
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")  

In [ ]:

con.execute(" copy (    SELECT  ST_GeomFromWKB(geometry) as  geometry, * exclude geometry  FROM 'data/aula4/spatial/cities.parquet' where country = 'UGA'  ) TO 'data/aula4/spatial/uga.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON')  ")
con.execute(" copy (    SELECT  ST_GeomFromWKB(geometry) as  geometry, * exclude geometry  FROM 'data/aula4/spatial/cities.parquet' where country = 'BRA'  ) TO 'data/aula4/spatial/bra.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON')  ")

In [ ]:
gpd.read_file('data/aula4/spatial/bra.geojson').explore()

In [ ]:
gpd.read_file('data/aula4/spatial/uga.geojson').explore()

## Full Text Search (FTS )
    - create_fts_index
    - schema com sufixo da tabela é criado Ex: fts_main_[nome da tabela]
    - Alteração de colunas = Manutenção do index (recriação do index :( )

In [ ]:
#create a table livros in duckdb 
con.execute("CREATE TABLE livros (id INTEGER, titulo VARCHAR, autor VARCHAR, descricao VARCHAR);")

In [ ]:
!pip install faker --quiet

In [ ]:
from faker import Faker


fake = Faker()

# Generate fake data
data = []
for _ in range(1000):
    id = fake.random_int(min=1, max=1000)
    titulo = fake.sentence(nb_words=5)
    autor = fake.name()
    descricao = fake.paragraph()
    data.append((id, titulo, autor, descricao))

# Insert data into the table
with con.cursor() as cursor:
    cursor.executemany("INSERT INTO livros (id, titulo, autor, descricao) VALUES (?, ?, ?, ?)", data)
con.commit()


In [ ]:
con.execute("SELECT * FROM livros").df()

In [ ]:
con.execute('''PRAGMA create_fts_index(
    'livros', 'id', 'titulo', 'autor', 'descricao'
);''')

In [ ]:
con.execute("INSTALL fts;")
con.execute("LOAD fts;") 

con.execute('''  SELECT *
FROM (
    SELECT *, fts_main_livros.match_bm25(
        id,
        'Party',
        fields := 'titulo'
    ) AS score
    FROM livros
) sq
WHERE score IS NOT NULL 
ORDER BY score DESC;   ''').df()

In [ ]:
con.execute(''' SELECT *
FROM (
    SELECT *, fts_main_livros.match_bm25(
        id,
        'strong politics'
    ) AS score
    FROM livros
) sq
WHERE score IS NOT NULL
ORDER BY score DESC; ''').df()

## Iceberg

    - [Referência](https://iceberg.apache.org/)
    - [Extensão](https://duckdb.org/docs/extensions/iceberg.html)
    - Somente Leitura no Duckdb (v.1.0.0)

In [ ]:
!wget https://duckdb.org/data/iceberg_data.zip
!unzip iceberg_data.zip
!rm -f iceberg_data.zip
!cp -r iceberg_data data/aula4/iceberg_data

In [ ]:
import duckdb
con = duckdb.connect()
con.execute("INSTALL iceberg;")
con.execute("LOAD iceberg;")
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [ ]:
%%sql 
install iceberg;
load iceberg;
SELECT * FROM iceberg_scan('data/aula4/iceberg/lineitem_iceberg', allow_moved_paths = true);

## Postgres

In [ ]:
import duckdb
con = duckdb.connect() 
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [ ]:
%%sql 

INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=postgres user=lord password=postgres123 host=127.0.0.1' AS db (TYPE POSTGRES);

In [ ]:
%%sql 
SHOW ALL TABLES;


In [ ]:
%%sql 

from 'data/aula4/spatial/cities.csv'  ;

In [ ]:
%%sql

CALL postgres_execute('db', 'CREATE TABLE cities (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    country VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    population INTEGER
);');



In [ ]:
%%sql

copy db.cities FROM  'data/aula4/spatial/cities.csv';

In [ ]:
%%sql

select * from db.cities;

In [ ]:
#mysql

In [ ]:
%%sql
INSTALL MYSQL;
LOAD MYSQL;
ATTACH 'host=localhost user=root   password=mysql123 database=lord_duckdb_mysql' AS mysqldb (TYPE MYSQL);
USE mysqldb;

In [ ]:
%%sql

install spatial;
load spatial;
 

In [ ]:
%%sql

CREATE TABLE mysqldb.cities (
    id INT  PRIMARY KEY,
    name VARCHAR(255),
    country VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    population INT
);

In [ ]:
%%sql 

insert into mysqldb.cities select *
  from ST_Read('data/aula4/spatial/cities.csv') where country = 'UGA' ;



In [ ]:
%%sql

from mysqldb.cities 